# MetaSpore Getting Started

MetaSpore is a machine learning platform, which provides a one-stop solution for data preprocessing, model training and online prediction.

In this article, we introduce the basic API of MetaSpore briefly.

## Prepare Data

We use the publicly available dataset [Terabyte Click Logs](https://labs.criteo.com/2013/12/download-terabyte-click-logs-2/) published by CriteoLabs as our demo dataset.

We sample the dataset with sampling rate 0.001 so that the running of the demo can finish quickly. More information about the demo dataset can be found in [MetaSpore Demo Dataset](https://ks3-cn-beijing.ksyuncs.com/dmetasoul-bucket/demo/criteo/index.html).

Execute the following cell to download the demo dataset into the working directory. Those data files take up about 2.1 GiB disk space and the downloading process may take sveral minutes. If the downloading fails, please refer to [MetaSpore Demo Dataset](https://ks3-cn-beijing.ksyuncs.com/dmetasoul-bucket/demo/criteo/index.html) and download the dataset manually.

In [1]:
# import metaspore
# metaspore.demo.download_dataset()

You can check the downloaded dataset by executing the following cell.

In [2]:
# !ls -l ${PWD}/data/

(Optional) To upload the dataset to your own s3 bucket:

1. Fill ``{YOUR_S3_BUCKET}`` and ``{YOUR_S3_PATH}`` with your preferred values in the following cell.
2. Uncomment the cell by removing the leading ``#`` character.
3. Execute the cell.

In [3]:
# YOUR_S3_BUCKET='s3://sagemaker-us-west-2-452145973879/datasets/CriteoLabs/'
# YOUR_S3_PATH='datasets/CriteoLabs'

In [4]:
# !aws s3 cp --recursive ${PWD}/data/ s3://sagemaker-us-west-2-452145973879/datasets/CriteoLabs/demo/data/

Alternatively, you can open a terminal by selecting the ``File`` -> ``New`` -> ``Terminal`` menu item and executing Bash commands in it.

You can check the uploaded dataset in your s3 bucket by uncommenting and executing the following cell.

In [5]:
# !aws s3 ls s3://sagemaker-us-west-2-452145973879/datasets/CriteoLabs/demo/data/
# !aws s3 ls s3://mv-mtg-di-for-poc-datalab/2024/06/14/00/

The ``schema`` directory contains configuration files and must also be uploaded to s3 so that the model can be trained in cluster environment. 

In [6]:
#!aws s3 cp --recursive ${PWD}/schema/ s3://sagemaker-us-west-2-452145973879/datasets/CriteoLabs/demo/schema/

In the rest of the article, we assume the demo dataset and schemas has been uploaded to `ROOT_DIR`.

In [7]:
# ROOT_DIR = 's3://sagemaker-us-west-2-452145973879/datasets/CriteoLabs/demo'
# ROOT_DIR = '.'
ROOT_DIR = 's3://mv-mtg-di-for-poc-datalab'

## Define the Model

We can define our neural network model by subclassing ``torch.nn.Module`` as usual PyTorch models. The following ``DemoModule`` class provides an example.

Compared to usual PyTorch models, the notable difference is the ``_sparse`` layer created by instantiating ``ms.EmbeddingSumConcat`` which takes an embedding size and paths of two text files. ``ms.EmbeddingSumConcat`` makes it possible to define large-scale sparse models in PyTorch, which is a distinguishing feature of MetaSpore.

The ``_schema_dir`` field is an s3 directory which makes it possible to use the ``DemoModule`` class in cluster environment.

In [8]:
# import torch
# import metaspore as ms

# class DemoModule(torch.nn.Module):
#     def __init__(self):
#         super().__init__()
#         self._embedding_size = 16
#         self._schema_dir = ROOT_DIR + '/schema/'
#         self._column_name_path = self._schema_dir + 'column_name_demo.txt'
#         self._combine_schema_path = self._schema_dir + 'combine_schema_demo.txt'
#         self._sparse = ms.EmbeddingSumConcat(self._embedding_size, self._column_name_path, self._combine_schema_path)
#         self._sparse.updater = ms.FTRLTensorUpdater()
#         self._sparse.initializer = ms.NormalTensorInitializer(var=0.01)
#         self._dense = torch.nn.Sequential(
#             ms.nn.Normalization(self._sparse.feature_count * self._embedding_size),
#             torch.nn.Linear(self._sparse.feature_count * self._embedding_size, 1024),
#             torch.nn.ReLU(),
#             torch.nn.Linear(1024, 512),
#             torch.nn.ReLU(),
#             torch.nn.Linear(512, 1),
#         )

#     def forward(self, x):
#         x = self._sparse(x)
#         x = self._dense(x)
#         return torch.sigmoid(x)

In [9]:
import metaspore as ms
import torch
import torch.nn as nn


def nansum(x):
    return torch.where(torch.isnan(x), torch.zeros_like(x), x).sum()


def log_loss(yhat, y):
    return nansum(-(y * (yhat + 1e-12).log() + (1 - y) *
                    (1 - yhat + 1e-12).log()))

# 自定义的主函数入口
class DNNModelMain(nn.Module):
    def __init__(self, ): # feature_config_file
        super().__init__()
        self._embedding_size = 16
        self._schema_dir = ROOT_DIR + '/schema/'
        self._column_name_path = self._schema_dir + 'column_name_mobivista.txt'
        self._combine_schema_path = self._schema_dir + 'combine_schema_mobivista.txt'
        # self.feature_config_file = feature_config_file  # TODO not used
        self._sparse = ms.EmbeddingSumConcat(
            self._embedding_size,
            combine_schema_source=self._column_name_path,
            combine_schema_file_path=self._combine_schema_path,
            # enable_feature_gen=True,
            # feature_config_file=feature_config_file,
            # enable_fgs=False
        )
        self._sparse.updater = ms.FTRLTensorUpdater(alpha=0.01)
        self._sparse.initializer = ms.NormalTensorInitializer(var=0.001)
        extra_attributes = {
            "enable_fresh_random_keep": True,
            "fresh_dist_range_from": 0, 
            "fresh_dist_range_to": 1000,
            "fresh_dist_range_mean": 950,
            "enable_feature_gen": True,
            "use_hash_code": False
        }
        self._sparse.extra_attributes = extra_attributes
        feature_count = self._sparse.feature_count
        feature_dim = self._sparse.feature_count * self._embedding_size

        self._gateEmbedding = GateEmbedding(feature_dim, feature_count, self._embedding_size)
        self._h1 = nn.Linear(feature_dim, 1024)
        self._h2 = FourChannelHidden(1024, 512)
        self._h3 = FourChannelHidden(512, 256)
        self._h4 = nn.Linear(256, 1)

        self._bn = ms.nn.Normalization(feature_dim, momentum=0.01, eps=1e-5, affine=True)
        self._zero = torch.zeros(1, 1)
        self.act0 = nn.Sigmoid()

    def forward(self, x):
        emb = self._sparse(x)
        bno = self._bn(emb)
        
        # print(f"self._sparse._data.type: {type(self._sparse._data)}, self._sparse._data.shape: {self._sparse._data.shape}") 
        # print(f"x.type: {type(x)}, x.shape: {x.shape}, x: {x}")
        # print(f"emb.type: {type(emb)}, emb.shape: {emb.shape}, ") # emb: {emb}
        # print(f"bno.type: {type(bno)}, bno.shape: {bno.shape}, ")  # bno: {bno}
        
        d = self._gateEmbedding(bno)
        o = self._h1(d)
        r, s1, s2, s3 = self._h2(o, self._zero, self._zero, self._zero)
        r, s1, s2, s3 = self._h3(r, s1, s2, s3)
        return self.act0(self._h4(r))


class FourChannelHidden(nn.Module):
    def __init__(self, in_size, out_size):
        super().__init__()
        self.wc2 = nn.Linear(int(in_size / 4), int(in_size / 4))
        self.wc3 = nn.Linear(int(in_size), int(in_size - in_size / 4 * 3))
        self.w = nn.Linear(int(in_size + int(in_size / 4) * 2) + int(in_size - int(in_size / 4) * 3) + 3, out_size)
        self.act1 = nn.Tanh()
        self.act = nn.ReLU()
        self.fl = int(in_size / 4)

    def forward(self, input, i1, i2, i3):
        f0 = input[:, :self.fl]
        f1 = input[:, self.fl:self.fl * 2]
        f2 = input[:, self.fl * 2:self.fl * 3]
        f3 = input[:, self.fl * 3:]

        c1 = self.act1(f0 * f1) * f1
        c2 = self.act1(self.wc2(f2) * f2)
        c3 = self.act1(f3 * self.wc3(input))

        s1 = torch.sum(c1, 1, True) + i1
        s2 = torch.sum(c2, 1, True) + i2
        s3 = torch.sum(c3, 1, True) + i3

        return self.act(self.w(torch.cat((input, c1, c2, c3, s1, s2, s3), 1))), s1, s2, s3


class GateEmbedding(nn.Module):
    def __init__(self, in_size, out_size, emb_size):
        super().__init__()
        self.layer1 = torch.nn.Linear(in_size, out_size)
        self.out_size = out_size
        self.emb_size = emb_size
        self.act2 = nn.Sigmoid()

    def forward(self, input):
        gate = self.act2(self.layer1(input))
        gate_reshape = torch.reshape(gate, (-1, self.out_size, 1))
        input_reshape = torch.reshape(input, (-1, self.out_size, self.emb_size))
        return (gate_reshape * input_reshape).reshape(-1, self.out_size * self.emb_size)

/opt/conda/envs/metaspore/lib/python3.8/site-packages/pyarrow/compute.py:196: RuntimeWarning: Python binding for StringBKDRHashFunctionOption not exposed
  warnings.warn("Python binding for {} not exposed"
/opt/conda/envs/metaspore/lib/python3.8/site-packages/pyarrow/compute.py:196: RuntimeWarning: Python binding for BKDRHashCombineFunctionOption not exposed
  warnings.warn("Python binding for {} not exposed"


Instantiating the ``DemoModule`` class to define our PyTorch model.

In [10]:
# module = DemoModule()
# module = DNNModelMain('schema/combine_schema_mobivista.txt')
module = DNNModelMain()

[WARN] 2024-07-16 02:51:43.621 STSAssumeRoleWithWebIdentityCredentialsProvider [140624245376832] Token file must be specified to use STS AssumeRole web identity creds provider.
loaded combine schema from combine schema file 's3://mv-mtg-di-for-poc-datalab/schema/combine_schema_mobivista.txt'
[2024-07-16 02:51:43.621] [info] [s3_sdk_filesys.cpp:357] Try to open S3 stream: s3://mv-mtg-di-for-poc-datalab/schema/combine_schema_mobivista.txt, read_only true
[2024-07-16 02:51:43.776] [info] [s3_sdk_filesys.cpp:380] Opened read-only stream for object: s3://mv-mtg-di-for-poc-datalab/schema/combine_schema_mobivista.txt with total length 2260
[2024-07-16 02:51:43.779] [info] [s3_sdk_filesys.cpp:419] Read S3 object s3://mv-mtg-di-for-poc-datalab/schema/combine_schema_mobivista.txt with size 2260 at position 0 larger than total size: 2260, change size to 2260
[2024-07-16 02:51:43.879] [info] [s3_sdk_filesys.cpp:413] Read S3 object s3://mv-mtg-di-for-poc-datalab/schema/combine_schema_mobivista.txt 

## Train the Model

To train our model, first we need to create a ``ms.PyTorchEstimator`` passing in several arguments including our PyTorch model ``module`` and the number of workers and servers.

``model_out_path`` specifies where to store the trained model.

``input_label_column_index`` specifies the column index of the label column in the dataset, which is ``0`` for the demo dataset.

In [11]:
model_out_path = ROOT_DIR + '/output/dev/model_out/'
estimator = ms.PyTorchEstimator(module=module,
                                worker_count=8,
                                server_count=8,
                                model_out_path=model_out_path,
                                experiment_name='0.1',
                                input_label_column_name='label',
                                training_epoches=1,
                                shuffle_training_dataset=True)

Next, we create a Spark session by calling ``ms.spark.get_session()`` and load the training dataset by call ``ms.input.read_s3_csv()``.

``delimiter`` specifies the column delimiter of the dataset, which is the TAB character ``'\t'`` for the demo dataset.

We also need to pass column names because the csv files do not contain headers.

In [12]:
column_names = []
with open(f'./schema/column_name_mobivista.txt', 'r') as f:
    for line in f:
        column_names.append(line.split(' ')[1].strip())
print(column_names)

['_11001', '_11002', '_11003', '_11004', '_11007', '_11008', '_11021', '_11022', '_11023', '_11024', '_11041', '_11042', '_11043', '_11044', '_11045', '_11046', '_11061', '_11062', '_11063', '_11064', '_11065', '_11066', '_11081', '_11082', '_11083', '_11084', '_11085', '_11086', '_11601', '_11602', '_11603', '_12001', '_12002', '_12003', '_12004', '_12005', '_12006', '_20001', '_20002', '_20003', '_20101', '_20102', '_20201', '_20202', '_20203', '_20204', '_20205', '_20206', '_20207', '_20208', '_20209', '_20210', '_30001', '_30002', '_30003', '_30004', '_30005', '_30006', '_30201', '_30202', '_30203', '_30204', '_30205', '_30206', '_30207', '_40001', '_40002', '_40003', '_40004', '_40005', '_40201', '_40202', '_40203', '_40204', '_40205', '_40206', '_40207', '_40208', '_40209', '_40210', '_40211', '_40212', '_40213', '_40214', '_40215', '_40231', '_40301', '_40302', '_40303', '_40304', '_40305', '_40306', '_40307', '_40321', '_40322', '_40323', '_40324', '_50801', '_50802', '_50805',

In [13]:
# train_dataset_path = ROOT_DIR + '/data/train/day_0_0.001_train.csv'

file_base_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'
num_files = 100
file_names = [f'part-{str(i).zfill(5)}-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc' for i in range(num_files)]
train_dataset_path = [file_base_path + file_name for file_name in file_names]

# train_dataset_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/part-00000-1e73cc51-9b17-4439-9d71-7d505df2cae3-c000.snappy.orc'
# train_dataset_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/14/00/'

spark_confs = {
    'spark.eventLog.enabled':'true',
    'spark.executor.memory': '20g',
    'spark.driver.memory': '10g',
}

spark_session = ms.spark.get_session(local=True,
                                     batch_size=200,
                                     worker_count=estimator.worker_count,
                                     server_count=estimator.server_count,
                                     log_level='WARN',
                                     spark_confs=spark_confs)

train_dataset = ms.input.read_s3_csv(spark_session, 
                                     train_dataset_path, 
                                     format='orc',
                                     shuffle=False,
                                     delimiter='\t', 
                                     multivalue_delimiter="\001", 
                                     column_names=column_names,
                                     multivalue_column_names=column_names[:-1])

# train_dataset = spark_session.read.orc(train_dataset_path)

# train_dataset.printSchema()

24/07/16 02:51:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


ignore shuffle


In [14]:
train_dataset.count()

1318351

In [15]:
# train_dataset = train_dataset.limit(10000)

Finally, we call the ``fit()`` method of ``ms.PyTorchEstimator`` to train our model. This will take several minutes and you can see the progress by looking at the output of the cell. The trained model is stored in ``model_out_path`` and the ``model`` variable.

In [16]:
model = estimator.fit(train_dataset)

[2024-07-16 02:52:24.365] [info] PS job with coordinator address 172.31.0.41:51327 started.
[2024-07-16 02:52:24.365] [info] PSRunner::RunPS: pid: 22031, tid: 22361, thread: 0x7fe52d780700
[2024-07-16 02:52:24.365] [info] PSRunner::RunPSCoordinator: pid: 22031, tid: 22361, thread: 0x7fe52d780700
[2024-07-16 02:52:24.366] [info] ActorProcess::Receiving: Coordinator pid: 22031, tid: 22365, thread: 0x7fe569798700


/opt/conda/envs/metaspore/lib/python3.8/site-packages/pyarrow/compute.py:196: RuntimeWarning: Python binding for StringBKDRHashFunctionOption not exposed
  warnings.warn("Python binding for {} not exposed"
/opt/conda/envs/metaspore/lib/python3.8/site-packages/pyarrow/compute.py:196: RuntimeWarning: Python binding for StringBKDRHashFunctionOption not exposed
  warnings.warn("Python binding for {} not exposed"
/opt/conda/envs/metaspore/lib/python3.8/site-packages/pyarrow/compute.py:196: RuntimeWarning: Python binding for BKDRHashCombineFunctionOption not exposed
  warnings.warn("Python binding for {} not exposed"
/opt/conda/envs/metaspore/lib/python3.8/site-packages/pyarrow/compute.py:196: RuntimeWarning: Python binding for BKDRHashCombineFunctionOption not exposed
  warnings.warn("Python binding for {} not exposed"
/opt/conda/envs/metaspore/lib/python3.8/site-packages/pyarrow/compute.py:196: RuntimeWarning: Python binding for StringBKDRHashFunctionOption not exposed
  warnings.warn("Pyt

[2024-07-16 02:52:25.803] [info] C[0]:9: The coordinator has connected to 8 servers and 8 workers.
PS Coordinator node C[0]:9 is ready.


[2024-07-16 02:52:25.804] [info] S[0]:10 has connected to others.
[2024-07-16 02:52:25.804] [info] S[2]:42 has connected to others.
[2024-07-16 02:52:25.804] [info] S[1]:26 has connected to others.
[2024-07-16 02:52:25.804] [info] W[1]:28 has connected to others.
[2024-07-16 02:52:25.804] [info] W[4]:76 has connected to others.
[2024-07-16 02:52:25.804] [info] W[5]:92 has connected to others.
[2024-07-16 02:52:25.804] [info] S[5]:90 has connected to others.
[2024-07-16 02:52:25.804] [info] S[4]:74 has connected to others.
[2024-07-16 02:52:25.804] [info] W[0]:12 has connected to others.
PS Server node S[2]:42 is ready.
PS Worker node W[1]:28 is ready.[2024-07-16 02:52:25.804] [info] S[6]:106 has connected to others.
[2024-07-16 02:52:25.804] [info] W[6]:108 has connected to others.
PS Server node S[1]:26 is ready.

PS Server node S[0]:10 is ready.[2024-07-16 02:52:25.804] [info] S[7]:122 has connected to others.

[2024-07-16 02:52:25.804] [info] S[3]:58 has connected to others.
[2024-0

shuffle df to partitions 16


...

2024-07-16 03:09:12.357 -- auc: 0.7835596172474558, Δauc: 0.7906722241254616, pcoc: 1.2842985712435375, Δpcoc: 0.9348099496629503, loss: 0.0005443605662675867, Δloss: 0.0005474042892456055, #instance: 1297168
2024-07-16 03:09:12.886 -- auc: 0.7836621918346321, Δauc: 0.8507416879795396, pcoc: 1.2839870054892186, Δpcoc: 1.066050222184923, loss: 0.0005442053123649028, Δloss: 0.00044351011514663695, #instance: 1299168
2024-07-16 03:09:14.178 -- auc: 0.7837302259700358, Δauc: 0.8277834699453552, pcoc: 1.28355673331476, Δpcoc: 1.0009934107462566, loss: 0.0005440984788491473, Δloss: 0.0004747011363506317, #instance: 1301168
2024-07-16 03:09:15.210 -- auc: 0.7838205573640724, Δauc: 0.837382474972403, pcoc: 1.2828854959046536, Δpcoc: 0.8904602174405698, loss: 0.0005440567586252274, Δloss: 0.0005169142484664917, #instance: 1303168
2024-07-16 03:09:19.983 -- auc: 0.7838610164170688, Δauc: 0.8095555642725454, pcoc: 1.2828085602259993, Δpcoc: 1.2204235883859487, loss: 0.0005438538621931378, Δloss: 

2024-07-16 03:09:27.925 -- auc: 0.7839065242436158, Δauc: 0.7606320717908701, pcoc: 1.2816720398513064, Δpcoc: 1.0844598900188098, loss: 0.0005435816696920206, Δloss: 0.0006334474876491779, #instance: 1311645
2024-07-16 03:09:27.935 -- auc: 0.7839545457010328, Δauc: 0.8335021097046413, pcoc: 1.2813592873247428, Δpcoc: 0.9497269332408905, loss: 0.0005437663865995976, Δloss: 0.0007431762699237086, #instance: 1312860
2024-07-16 03:09:27.945 -- auc: 0.7840202067993857, Δauc: 0.8535724452035297, pcoc: 1.2809751588197948, Δpcoc: 0.8732739679515362, loss: 0.0005437249852831285, Δloss: 0.0004984675894966729, #instance: 1314061
2024-07-16 03:09:27.954 -- auc: 0.7840727525444542, Δauc: 0.87704802259887, pcoc: 1.2811067188550616, Δpcoc: 1.5606367111206054, loss: 0.0005435780537432858, Δloss: 0.0003422464518900087, #instance: 1315020
2024-07-16 03:09:27.964 -- auc: 0.7840840785750797, Δauc: 0.7954792658055745, pcoc: 1.2806182923373604, Δpcoc: 0.8912690937519073, loss: 0.000543572396681654, Δloss: 

saving model to s3://mv-mtg-di-for-poc-datalab/output/dev/model_out/
saving model to s3://mv-mtg-di-for-poc-datalab/output/dev/model_out/saving model to s3://mv-mtg-di-for-poc-datalab/output/dev/model_out/saving model to s3://mv-mtg-di-for-poc-datalab/output/dev/model_out/


saving model to s3://mv-mtg-di-for-poc-datalab/output/dev/model_out/
saving model to s3://mv-mtg-di-for-poc-datalab/output/dev/model_out/
saving model to s3://mv-mtg-di-for-poc-datalab/output/dev/model_out/saving model to s3://mv-mtg-di-for-poc-datalab/output/dev/model_out/

Get aws endpoint from env: 
[WARN] 2024-07-16 03:09:28.435 STSAssumeRoleWithWebIdentityCredentialsProvider [140237118707520] Token file must be specified to use STS AssumeRole web identity creds provider.
[2024-07-16 03:09:28.435] [info] [s3_sdk_filesys.cpp:357] Try to open S3 stream: s3://mv-mtg-di-for-poc-datalab/output/dev/model_out/_sparse__sparse_meta.json, read_only false
Get aws endpoint from env:  (0 + 8) / 8][Stage 20:>                

[2024-07-16 03:13:40.911] [info] C[0]:9 has stopped.
[2024-07-16 03:13:40.919] [info] PS job with coordinator address 172.31.0.41:51327 stopped.


[2024-07-16 03:13:41.190] [info] PS job with coordinator address 172.31.0.41:51327 stopped.
[2024-07-16 03:13:41.191] [info] PS job with coordinator address 172.31.0.41:51327 stopped.
[2024-07-16 03:13:41.204] [info] PS job with coordinator address 172.31.0.41:51327 stopped.
[2024-07-16 03:13:41.228] [info] PS job with coordinator address 172.31.0.41:51327 stopped.
[2024-07-16 03:13:41.231] [info] PS job with coordinator address 172.31.0.41:51327 stopped.
[2024-07-16 03:13:41.235] [info] PS job with coordinator address 172.31.0.41:51327 stopped.
[2024-07-16 03:13:41.243] [info] PS job with coordinator address 172.31.0.41:51327 stopped.
[2024-07-16 03:13:41.244] [info] PS job with coordinator address 172.31.0.41:51327 stopped.


## Evaluate the Model

To evaluate our model, we use the ``ms.input.read_s3_csv()`` function again to load the test dataset, passing in the column delimiter ``'\t'``.

In [17]:
# test_dataset_path = ROOT_DIR + '/data/test/day_1_0.001_test.csv'
# test_dataset = ms.input.read_s3_csv(spark_session, test_dataset_path, delimiter='\t', column_names=column_names)

test_dataset_path = 's3://mv-mtg-di-for-poc-datalab/2024/06/15/00/part-00000-f79b9ee6-aaf5-4117-88d5-44eea69dcea3-c000.snappy.orc'
# test_dataset = spark_session.read.orc(test_dataset_path)
test_dataset = ms.input.read_s3_csv(spark_session, 
                                     test_dataset_path, 
                                     format='orc',
                                     delimiter='\t', 
                                     multivalue_delimiter="\001", 
                                     column_names=column_names,
                                     multivalue_column_names=column_names[:-1])
# test_dataset.printSchema()

ignore shuffle


Next, we call the ``model.transform()`` method to transform the test dataset, which will add a column named ``rawPrediction`` to the test dataset representing the predicted labels. For ease of integration with Spark MLlib, ``model.transform()`` will also add a column named ``label`` to the test dataset representing the actual labels.

Like the training process, this will take several minutes and you can see the progress by looking at the output of the cell. The transformed test dataset is stored in the ``result`` variable.

In [18]:
result = model.transform(test_dataset)

[2024-07-16 03:13:44.509] [info] PS job with coordinator address 172.31.0.41:59757 started.
[2024-07-16 03:13:44.509] [info] PSRunner::RunPS: pid: 22031, tid: 23805, thread: 0x7fe557f91700
[2024-07-16 03:13:44.509] [info] PSRunner::RunPSCoordinator: pid: 22031, tid: 23805, thread: 0x7fe557f91700
[2024-07-16 03:13:44.510] [info] ActorProcess::Receiving: Coordinator pid: 22031, tid: 23808, thread: 0x7fe569798700
[2024-07-16 03:13:44.534] [info] C[0]:9: The coordinator has connected to 8 servers and 8 workers.
PS Coordinator node C[0]:9 is ready.


[2024-07-16 03:13:44.529] [info] PS job with coordinator address 172.31.0.41:59757 started.
[2024-07-16 03:13:44.529] [info] PSRunner::RunPS: pid: 22375, tid: 23820, thread: 0x7f8b52f4b700
[2024-07-16 03:13:44.529] [info] PSRunner::RunPSWorker: pid: 22375, tid: 23820, thread: 0x7f8b52f4b700
ps agent registered for process 22375 thread 0x7f8b7fa51740
[2024-07-16 03:13:44.530] [info] PS job with coordinator address 172.31.0.41:59757 started.
[2024-07-16 03:13:44.530] [info] PSRunner::RunPS: pid: 22406, tid: 23824, thread: 0x7f8b52f4b700
[2024-07-16 03:13:44.530] [info] PSRunner::RunPSWorker: pid: 22406, tid: 23824, thread: 0x7f8b52f4b700
[2024-07-16 03:13:44.530] [info] PS job with coordinator address 172.31.0.41:59757 started.
[2024-07-16 03:13:44.530] [info] PSRunner::RunPS: pid: 22380, tid: 23825, thread: 0x7f8b52f4b700
[2024-07-16 03:13:44.530] [info] PSRunner::RunPSWorker: pid: 22380, tid: 23825, thread: 0x7f8b52f4b700
ps agent registered for process 22406 thread 0x7f8b7fa51740
ps a

_11001
[2024-07-16 03:13:44.771] [info] add expr bkdr_hash(_11001, StringBKDRHashFunctionOption::name=_11001)
_11002
[2024-07-16 03:13:44.771] [info] add expr bkdr_hash(_11002, StringBKDRHashFunctionOption::name=_11002)
_11003
[2024-07-16 03:13:44.771] [info] add expr bkdr_hash(_11003, StringBKDRHashFunctionOption::name=_11003)
_11004
[2024-07-16 03:13:44.771] [info] add expr bkdr_hash(_11004, StringBKDRHashFunctionOption::name=_11004)
_11007
[2024-07-16 03:13:44.771] [info] add expr bkdr_hash(_11007, StringBKDRHashFunctionOption::name=_11007)
_11008
[2024-07-16 03:13:44.771] [info] add expr bkdr_hash(_11008, StringBKDRHashFunctionOption::name=_11008)
_11021
[2024-07-16 03:13:44.771] [info] add expr bkdr_hash(_11021, StringBKDRHashFunctionOption::name=_11021)
_11022
[2024-07-16 03:13:44.771] [info] add expr bkdr_hash(_11022, StringBKDRHashFunctionOption::name=_11022)
_11023
[2024-07-16 03:13:44.771] [info] add expr bkdr_hash(_11023, StringBKDRHashFunctionOption::name=_11023)
_11024
[20

2024-07-16 03:17:34.419 -- auc: 0.7817412414829121, Δauc: 0.7817412414829121, pcoc: 1.4757429549568577, Δpcoc: 1.4757429549568577, loss: 0.0004228756427764893, Δloss: 0.0004228756427764893, #instance: 2000
2024-07-16 03:17:36.714 -- auc: 0.8215325670498084, Δauc: 0.854364807007223, pcoc: 1.3041277212255142, Δpcoc: 1.165374979059747, loss: 0.00043420791625976564, Δloss: 0.000445540189743042, #instance: 4000
2024-07-16 03:17:37.440 -- auc: 0.8234624735562502, Δauc: 0.8248358974358975, pcoc: 1.2576682479293257, Δpcoc: 1.1786871433258057, loss: 0.00045395630598068235, Δloss: 0.0004934530854225159, #instance: 6000
2024-07-16 03:17:40.085 -- auc: 0.8258774105906745, Δauc: 0.8341541417384114, pcoc: 1.2729164732378082, Δpcoc: 1.3219286260150729, loss: 0.0004485385343432426, Δloss: 0.00043228521943092344, #instance: 8000


2024-07-16 03:17:43.131 -- auc: 0.8263654279279279, Δauc: 0.8286422061229259, pcoc: 1.217808922816967, Δpcoc: 1.040462806008079, loss: 0.00046418623328208923, Δloss: 0.0005267770290374756, #instance: 10000


[2024-07-16 03:17:46.713] [info] W[0]:12 has stopped.                           
[2024-07-16 03:17:46.713] [info] W[2]:44 has stopped.
[2024-07-16 03:17:46.713] [info] W[1]:28 has stopped.
[2024-07-16 03:17:46.713] [info] W[4]:76 has stopped.
[2024-07-16 03:17:46.713] [info] W[3]:60 has stopped.
[2024-07-16 03:17:46.713] [info] W[5]:92 has stopped.
[2024-07-16 03:17:46.713] [info] W[7]:124 has stopped.
[2024-07-16 03:17:46.713] [info] W[6]:108 has stopped.
[2024-07-16 03:17:46.713] [info] S[2]:42 has stopped.
[2024-07-16 03:17:46.713] [info] S[0]:10 has stopped.
[2024-07-16 03:17:46.713] [info] S[1]:26 has stopped.
[2024-07-16 03:17:46.713] [info] S[5]:90 has stopped.
[2024-07-16 03:17:46.713] [info] S[4]:74 has stopped.
[2024-07-16 03:17:46.713] [info] S[3]:58 has stopped.
[2024-07-16 03:17:46.713] [info] S[7]:122 has stopped.
[2024-07-16 03:17:46.713] [info] S[6]:106 has stopped.
[2024-07-16 03:17:46.714] [info] PS job with coordinator address 172.31.0.41:59757 stopped.
[2024-07-16 0

2024-07-16 03:17:46.607 -- auc: 0.8211806566109294, Δauc: 0.7903020892151327, pcoc: 1.1986445928142018, Δpcoc: 1.0927844842274983, loss: 0.0004772909062956444, Δloss: 0.0005612952204851004, #instance: 11560
2024-07-16 03:17:46.618 -- auc: 0.8237797902561834, Δauc: 0.8493577981651377, pcoc: 1.1813690301619078, Δpcoc: 1.023585557937622, loss: 0.0004819464279274083, Δloss: 0.0005299980619124004, #instance: 12680
2024-07-16 03:17:46.628 -- auc: 0.8244418633400065, Δauc: 0.8313761467889909, pcoc: 1.1717252688493558, Δpcoc: 1.0740018208821616, loss: 0.00048586983611618263, Δloss: 0.0005302884216819491, #instance: 13800
2024-07-16 03:17:46.664 -- auc: 0.8244418633400065, Δauc: 1.0, pcoc: 1.1717252688493558, Δpcoc: nan, loss: 0.00048586983611618263, Δloss: nan, #instance: 13800
2024-07-16 03:17:46.674 -- auc: 0.8244418633400065, Δauc: 1.0, pcoc: 1.1717252688493558, Δpcoc: nan, loss: 0.00048586983611618263, Δloss: nan, #instance: 13800
2024-07-16 03:17:46.684 -- auc: 0.8244418633400065, Δauc: 1

[2024-07-16 03:17:46.876] [info] PS job with coordinator address 172.31.0.41:59757 stopped.
[2024-07-16 03:17:46.889] [info] PS job with coordinator address 172.31.0.41:59757 stopped.
[2024-07-16 03:17:46.889] [info] PS job with coordinator address 172.31.0.41:59757 stopped.
[2024-07-16 03:17:46.890] [info] PS job with coordinator address 172.31.0.41:59757 stopped.
[2024-07-16 03:17:46.891] [info] PS job with coordinator address 172.31.0.41:59757 stopped.
[2024-07-16 03:17:46.892] [info] PS job with coordinator address 172.31.0.41:59757 stopped.
[2024-07-16 03:17:46.900] [info] PS job with coordinator address 172.31.0.41:59757 stopped.
[2024-07-16 03:17:46.912] [info] PS job with coordinator address 172.31.0.41:59757 stopped.


``result`` is a normal PySpark DataFrame and can be inspected by its methods.

In [19]:
result.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------

In [20]:
result_pd = result.toPandas()
result_pd[result_pd['label']==0].head()


,_11001,_11002,_11003,_11004,_11007,_11008,_11021,_11022,_11023,_11024,...,_80144,_80145,_80146,_80147,_80148,_80149,_80150,_80151,label,rawPrediction
0,"[120248835527556541700.6931471824645996, 1493...","[141772368161457520630.6931471824645996, 1493...",[1187569319056953.4339871406555176],[1187569319062941.3862943649291992],"[120248835527556541700.6931471824645996, 1417...",[1187569319390603.4339871406555176],[93351669474747412440.6931471824645996],[57474464586887399740.6931471824645996],[1187569318903570.6931471824645996],[1187569318902660.6931471824645996],...,"[108444874906803988331.0, 1442976750699403878...",[120778770410835650001.0],[137161293658021960231.0],[177868718758286701041.0],[120778770410834821051.0],[120778770410834530021.0],[11264345888808339641.0],[120778770410778302591.0],0.0,0.155403
1,"[10787832899110671220.6931471824645996, 11267...",[],[1187569319056953.4339871406555176],[],"[118330750822017570580.6931471824645996, 1272...",[1187569319390602.995732307434082],"[10787832899110671220.6931471824645996, 11960...","[112678252360946792430.6931471824645996, 1250...",[1187569318903571.6094379425048828],[1187569318902661.945910096168518],...,"[10159436557395306301.0, 11545538683475017864...","[130216112555249546711.0, 2597973395502793715...",[62423217256752089271.0],[130216112555249132891.0],"[130216112555249009521.0, 1392697776517784263...","[130216112555249051311.0, 1751367607848644577...",[62423217256694666601.0],[130216112555210877641.0],0.0,0.192204
2,"[118074487017061505650.6931471824645996, 1183...","[103441021300152843870.6931471824645996, 1069...",[1187569319056952.5649492740631104],[1187569319062942.4849066734313965],"[103441021300152843870.6931471824645996, 1069...",[1187569319390603.1354942321777344],[],[56781507164361920530.6931471824645996],[],[1187569318902660.6931471824645996],...,"[127059589628045409081.0, 1770294181161403403...",[124069974375180026861.0],[20143758067925801991.0],[127059589628044381051.0],"[127059589628044259601.0, 1770294181161424728...",[124069974375179531441.0],[20143758068203763001.0],[127059589627985483241.0],0.0,0.009473
3,"[100526117827408219330.6931471824645996, 1155...","[101574285697699037070.6931471824645996, 1084...",[1187569319056954.234106540679932],[1187569319062945.164785861968994],"[100526117827408219330.6931471824645996, 1015...",[1187569319390605.283203601837158],"[132182422960092266980.6931471824645996, 1683...","[103057321266917028030.6931471824645996, 1031...",[1187569318903571.0986123085021973],[1187569318902663.97029185295105],...,"[106001042736559328931.0, 1103157403546153005...","[37683130965266694421.0, 6048009056449158698...",[63340032098683249861.0],[60480090564491828621.0],"[103576476166836788421.0, 1063744915184569113...","[121314994446557680821.0, 3768313096526881540...",[91696515358083713801.0],[83815087634930800461.0],0.0,0.086590
4,"[115633972886692014871.3862943649291992, 1231...","[115633972886692014870.6931471824645996, 1225...",[1187569319056953.178053855895996],[1187569319062942.7725887298583984],"[115633972886692014871.6094379425048828, 1225...",[1187569319390603.5835189819335938],[57474464586887399740.6931471824645996],[73737875405770893260.6931471824645996],[1187569318903570.6931471824645996],[1187569318902660.6931471824645996],...,"[17575797228106840461.0, 8425347615015788875...",[120362844306992554561.0],[85052345276137848811.0],[17575797228106177231.0],[63361388077108851291.0],[63361388077108892421.0],[45775881559041977191.0],[63361388079422640031.0],0.0,0.000248


In [21]:
result_pd[result_pd['label']==1].head()

,_11001,_11002,_11003,_11004,_11007,_11008,_11021,_11022,_11023,_11024,...,_80144,_80145,_80146,_80147,_80148,_80149,_80150,_80151,label,rawPrediction
63,"[104620418056146081330.6931471824645996, 1062...",[],[1187569319056954.753590106964111],[],"[104620418056146081330.6931471824645996, 1062...",[1187569319390604.430816650390625],"[127892341845370442530.6931471824645996, 1563...","[110950711939454100660.6931471824645996, 1183...",[1187569318903571.7917594909667969],[1187569318902663.367295742034912],...,"[105933614083154295231.0, 1062522390975181368...","[129596072363121941871.0, 1527917215876760072...",[156051290013057733801.0],[15279172158766531601.0],"[105933614083153836971.0, 1072649206291084363...","[128620735839288783061.0, 1295960723631221070...",[156051290012978026391.0],[15279172158731226111.0],1.0,0.096737
113,"[142318104563597162540.6931471824645996, 3526...","[142318104563597162541.3862943649291992, 3610...",[1187569319056953.9889841079711914],[1187569319062945.459585666656494],"[142318104563597162541.6094379425048828, 3526...",[1187569319390605.568344593048096],[36107841458994229780.6931471824645996],"[39630214814404936760.6931471824645996, 40457...",[1187569318903570.6931471824645996],[1187569318902662.5649492740631104],...,"[106061919320742175361.0, 1060631373873155932...",[136917921933945535191.0],[50928361379492542301.0],[136917921933944791131.0],"[106081817414152852661.0, 1369179219339449980...",[136917921933945075631.0],[50928361379415778851.0],[136917921933865744841.0],1.0,0.030930
132,"[100101933848291012220.6931471824645996, 1016...","[107734475477308076950.6931471824645996, 1114...",[1187569319056955.288267135620117],[1187569319062944.574710845947266],"[100101933848291012220.6931471824645996, 1016...",[1187569319390605.463831901550293],"[172840892160596556460.6931471824645996, 1730...","[107734475477308076951.0986123085021973, 1102...",[1187569318903572.397895336151123],[1187569318902663.6888794898986816],...,"[110204691836615960801.0, 1122047231687370522...",[52646567680003071521.0],[67103645266423977451.0],[135869932307155051571.0],"[110204691836614811401.0, 1358699323071532923...","[135869932307153329031.0, 6425259470847241092...",[183937062994583982991.0],[64252594708416056731.0],1.0,0.100108
315,[],[],[],[],[],[],[],[],[],[],...,"[109607787170272620181.0, 1107201198244415906...",[126855478711768204071.0],[26881475138864686101.0],[114504604610535394681.0],[126855478711768645441.0],[126855478711767703551.0],[11264281730343989061.0],[126855478711711187481.0],1.0,0.007977
329,"[103767651444137829250.6931471824645996, 1039...","[100978332819887972301.0986123085021973, 1019...",[1187569319056954.672828674316406],[1187569319062944.9972124099731445],"[101916632624280216970.6931471824645996, 1071...",[1187569319390604.9628448486328125],"[103918480814268574851.3862943649291992, 1064...","[100978332819887972301.0986123085021973, 1037...",[1187569318903573.4339871406555176],[1187569318902664.8978400230407715],...,"[116579843032178911731.0, 1191277981337401093...","[145517052396087058301.0, 1552937270282783357...",[50088960850490903331.0],[155293727028277601971.0],"[112179979732406231741.0, 1191277981337422315...","[119127798133742272031.0, 1455170523960872540...",[61040712848087343141.0],[119127798133661016761.0],1.0,0.060428


Finally, we use ``pyspark.ml.evaluation.BinaryClassificationEvaluator`` to compute test AUC.

In [22]:
import pyspark
evaluator = pyspark.ml.evaluation.BinaryClassificationEvaluator()
test_auc = evaluator.evaluate(result)
print('test_auc: %g' % test_auc)

test_auc: 0.82442


When all computations are done, we should call the ``stop()`` method of ``spark_session`` to make sure all the resources are released.

In [23]:
# spark_session.stop()

## Summary

We illustrated how to train and evaluate neural network model in MetaSpore. Users familiar with PyTorch and Spark MLlib should get started easily, which is the design goal of MetaSpore.